# Test gym

In [1]:
import gym

env = gym.make('CartPole-v0')

[2017-09-30 17:30:04,705] Making new env: CartPole-v0


In [2]:
obs = env.reset()
obs

array([ 0.03666635, -0.03884302,  0.01092027, -0.03935339])

In [3]:
env.render()

In [4]:
env.action_space

Discrete(2)

In [ ]:
action = 1
obs, reward, done, info = env.step(action)
obs

In [ ]:
reward

In [ ]:
done

In [ ]:
info

# Network

In [5]:
import tensorflow as tf

## Construction phase

In [6]:
n_inputs = 4
n_hidden = 4
n_outputs = 1
initializer = tf.contrib.layers.variance_scaling_initializer()

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs, activation=None, kernel_initializer=initializer)

outputs = tf.nn.sigmoid(logits)
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()


## Rewads

In [7]:
import numpy as np

def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    comulative_rewards = 0
    for step in reversed(range(len(rewards))):
        comulative_rewards = rewards[step] + comulative_rewards * discount_rate
        discounted_rewards[step] = comulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

# test
discount_rewards([10, 0, -50], discount_rate=0.8)

array([-22., -40., -50.])

In [8]:
discount_and_normalize_rewards([[10, 0, -50], [10, 20]], discount_rate=0.8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([ 1.26665318,  1.0727777 ])]

## Policy Network Training

In [12]:
n_iterations = 250
n_max_steps = 1000
n_games_per_update = 10
save_iterations = 10
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = []
        all_gradients = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            
            obs = env.reset()
            for step in range(n_max_steps):
                observation = np.array(obs.reshape(1, n_inputs))
                print(observation)
                action_val, gradients_val = sess.run(
                    [action, gradients],
                    feed_dict={X: observation})
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=0.8)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean(
                [reward * all_gradients[game_index][step][var_index]
                    for game_index, rewards in enumerate(all_rewards)
                    for step, reward in enumerate(rewards)], axis=0)
            feed_dict[grad_placeholder] = mean_gradients
            sess.run(training_op, feed_dict=feed_dict)
            if iteration % save_iterations == 0:
                saver.save(sess, './my_policy_net_pg.ckpt')

[[ 0.03260074 -0.04385996  0.02753348  0.04456524]]
[[ 0.03172354 -0.23936569  0.02842478  0.3458065 ]]
[[ 0.02693623 -0.4348802   0.03534091  0.64731556]]
[[ 0.01823863 -0.240268    0.04828723  0.36596761]]
[[ 0.01343327 -0.4360417   0.05560658  0.6734768 ]]
[[ 0.00471243 -0.24173491  0.06907611  0.39880702]]
[[ -1.22265249e-04  -4.37765326e-01   7.70522548e-02   7.12443631e-01]]
[[-0.00887757 -0.63386481  0.09130113  1.02835106]]
[[-0.02155487 -0.83007548  0.11186815  1.34824647]]
[[-0.03815638 -1.02641142  0.13883308  1.6737302 ]]
[[-0.05868461 -1.22284547  0.17230768  2.00623212]]
[[ 0.04319331  0.02161441 -0.04115305 -0.0164816 ]]
[[ 0.04362559 -0.17289393 -0.04148269  0.2629383 ]]
[[ 0.04016772  0.02279484 -0.03622392 -0.04253474]]
[[ 0.04062361 -0.17178945 -0.03707462  0.23850272]]
[[ 0.03718782 -0.36636268 -0.03230456  0.51926456]]
[[ 0.02986057 -0.17080118 -0.02191927  0.21657948]]
[[ 0.02644455 -0.36560304 -0.01758768  0.50226831]]
[[ 0.01913249 -0.56047272 -0.00754231  0.789

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [4]
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[4], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/home/xpitfire/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/xpitfire/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/xpitfire/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/xpitfire/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/xpitfire/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/xpitfire/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.p